In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/root/mmdet3d


In [3]:
import numpy as np
from scipy.spatial.transform import Rotation
from typing import Any, Dict, List, Optional, Tuple
import os
import torch
import random
from mmcv import Config
from torchpack.utils.config import configs
from mmdet3d.utils import recursive_eval
from mmdet3d.datasets import build_dataloader, build_dataset
import copy
from pytorch3d.ops import box3d_overlap

def calc_full_iou(box_a, box_b) -> float:
    box_a_size = np.array(box_a["size"])  # wlh
    box_b_size = np.array(box_b["size"])  # wlh

    assert all(box_a_size > 0), "Error: box_a sizes must be >0."
    assert all(box_b_size > 0), "Error: box_b sizes must be >0."

    box_a_min_z = box_a["translation"][2] - box_a_size[2] / 2
    box_a_max_z = box_a["translation"][2] + box_a_size[2] / 2

    box_b_min_z = box_b["translation"][2] - box_b_size[2] / 2
    box_b_max_z = box_b["translation"][2] + box_b_size[2] / 2

    min_z = max(box_a_min_z, box_b_min_z)
    max_z = min(box_a_max_z, box_b_max_z)

    print("max_z", max_z)
    print("min_z", min_z)

    height_intersection = max(0, max_z - min_z)

    box_a_ground_bbox_coords = calculate_ground_bbox_coords(box_a)
    box_b_ground_bbox_coords = calculate_ground_bbox_coords(box_b)

    area_intersection = calculate_area_of_intersection(
        box_a_ground_bbox_coords, box_b_ground_bbox_coords
    )

    print("area_intersection", area_intersection)
    print("height_intersection", height_intersection)

    intersection = height_intersection * area_intersection

    print("intersection", intersection)

    box_a_volume = np.prod(box_a_size)
    box_b_volume = np.prod(box_b_size)

    print("box_a_volume", box_a_volume)
    print("box_b_volume", box_b_volume)

    union = box_a_volume + box_b_volume - intersection

    print("union", union)

    print("iou-raw", intersection / union)

    iou = np.clip(intersection / union, 0, 1)

    return iou

def check_orthogonal(a, b, c):
    """Check that vector (b - a) is orthogonal to the vector (c - a)."""
    return np.isclose((b[0] - a[0]) * (c[0] - a[0]) + (b[1] - a[1]) * (c[1] - a[1]), 0)

def calculate_area_of_intersection(
    box_a_coords: List[Tuple[float, float]], box_b_coords: List[Tuple[float, float]]
) -> float:
    """
    Calculate the area of intersection between two polygons.
    :param box_a_coords: List of coordinates of the first polygon.
    :param box_b_coords: List of coordinates of the second polygon.
    :return: Area of intersection.
    """
    assert len(box_a_coords) == len(
        box_b_coords
    ), "Error: Polygons must have same number of points."

    area = 0
    for i in range(len(box_a_coords) - 1):
        area += triangle_area(box_a_coords[i], box_a_coords[i + 1], box_b_coords[i + 1])
        area += triangle_area(box_a_coords[i], box_b_coords[i + 1], box_b_coords[i])
    print("area", area)
    return area

def triangle_area(a, b, c):
    """Calculate the area of a triangle."""
    return abs((a[0] * (b[1] - c[1]) + b[0] * (c[1] - a[1]) + c[0] * (a[1] - b[1])) / 2)

def calculate_ground_bbox_coords(box) -> List[Tuple[float, float]]:
    center = box["translation"][:2]
    width = box["size"][0]  # wlh
    length = box["size"][1]  # wlh

    center_x = center[0]
    center_y = center[1]

    rotation_matrix = Rotation.from_euler("z", box["rotation"]).as_matrix()[0:2, 0:2]
    cos_angle = rotation_matrix[0, 0]
    sin_angle = rotation_matrix[1, 0]

    point_0_x = center_x + length / 2 * cos_angle + width / 2 * sin_angle
    point_0_y = center_y + length / 2 * sin_angle - width / 2 * cos_angle

    point_1_x = center_x + length / 2 * cos_angle - width / 2 * sin_angle
    point_1_y = center_y + length / 2 * sin_angle + width / 2 * cos_angle

    point_2_x = center_x - length / 2 * cos_angle - width / 2 * sin_angle
    point_2_y = center_y - length / 2 * sin_angle + width / 2 * cos_angle

    point_3_x = center_x - length / 2 * cos_angle + width / 2 * sin_angle
    point_3_y = center_y - length / 2 * sin_angle - width / 2 * cos_angle

    point_0 = point_0_x, point_0_y
    point_1 = point_1_x, point_1_y
    point_2 = point_2_x, point_2_y
    point_3 = point_3_x, point_3_y

    assert check_orthogonal(point_0, point_1, point_3)
    assert check_orthogonal(point_1, point_0, point_2)
    assert check_orthogonal(point_2, point_1, point_3)
    assert check_orthogonal(point_3, point_0, point_2)

    return [
        (point_0_x, point_0_y),
        (point_1_x, point_1_y),
        (point_2_x, point_2_y),
        (point_3_x, point_3_y),
        (point_0_x, point_0_y),
    ]

def output_to_box_dict(box3d, scores, labels):
    """Convert the output to the box class in the nuScenes.

    Args:
        detection (dict): Detection results.

            - boxes_3d (:obj:`BaseInstance3DBoxes`): Detection bbox.
            - scores_3d (torch.Tensor): Detection scores.
            - labels_3d (torch.Tensor): Predicted box labels.

    Returns:
        list[:obj:`dict`]: List of standard box dicts.
    """

    box_gravity_center = box3d.gravity_center.numpy()
    box_dims = box3d.dims.numpy()
    box_yaw = box3d.yaw.numpy()
    # TODO: check whether this is necessary
    # with dir_offset & dir_limit in the head

    box_list = []
    for i in range(len(box3d)):
        box = {
            "center": np.array(box_gravity_center[i]),
            "wlh": np.array(box_dims[i]),
            "orientation": box_yaw[i],
            "label": int(labels[i]) if not np.isnan(labels[i]) else labels[i],
            "score": float(scores[i]) if not np.isnan(scores[i]) else scores[i],
            "name": None,
        }
        box_list.append(box)
        
    return box_list

In [4]:
config_path = "configs/tumtraf-i/baseline/transfusion/lidar/voxelnet-1600g-0xy1-0z20-gtp15.yaml"
assert os.path.exists(config_path), "config not found"

# load config
configs.load(config_path, recursive=True)
cfg = Config(recursive_eval(configs), filename=config_path)

In [5]:
random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.manual_seed(cfg.seed)

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 4
dataset = build_dataset(cfg.data.train)

idx = 0
input_dict = dataset.dataset.get_data_info(idx)
dataset.dataset.pre_pipeline(input_dict)
example = dataset.dataset.pipeline(input_dict)

gt_bboxes_3d = example["gt_bboxes_3d"].data
gt_labels_3d = example["gt_labels_3d"].data.numpy()
gt_scores =  np.ones_like(gt_labels_3d)

ts = example["metas"].data["timestamp"]
box = output_to_box_dict(gt_bboxes_3d, gt_scores, gt_labels_3d)

box_idx = 0
name = dataset.dataset.CLASSES[box[box_idx]["label"]]

box_a_translation = box[box_idx]["center"].tolist()
box_a_size = box[box_idx]["wlh"].tolist()
box_a_rotation = box[box_idx]["orientation"]

print("og box_a_translation", box_a_translation)
print("og box_a_size", box_a_size)

box_a = dict(
    timestamp=ts,
    translation=box_a_translation,
    size=box_a_size,
    rotation=box_a_rotation,
    detection_name=name,
    detection_score=box[box_idx]["score"],
)

box_b_translation = box[box_idx]["center"].tolist()
box_b_size = box[box_idx]["wlh"].tolist()
box_b_rotation = box[box_idx]["orientation"]

box_b_translation[0] += 0 # translation works
box_b_translation[1] += 0 # translation works
# box_b_size[1] += 0.1 # size works
# box_b_rotation += 0.2 # rotation works

box_b = dict(
    timestamp=ts,
    translation=box_b_translation,
    size=box_b_size,
    rotation=box_b_rotation,
    detection_name=name,
    detection_score=box[box_idx]["score"],
)

iou = 1 - calc_full_iou(box_a, box_b)
print("iou", iou)


og box_a_translation [42.36349105834961, 4.04207181930542, -8.299816131591797]
og box_a_size [3.937201499938965, 1.8238351345062256, 1.4393945932388306]
max_z -7.580118834972382
min_z -9.019513428211212
area 0.0
area_intersection 0.0
height_intersection 1.4393945932388306
intersection 0.0
box_a_volume 10.336013946434097
box_b_volume 10.336013946434097
union 20.672027892868194
iou-raw 0.0
iou 1.0


In [6]:
ba = example["gt_bboxes_3d"].data.corners
bb = copy.deepcopy(ba)

In [7]:
example["gt_bboxes_3d"].data.tensor.shape

torch.Size([12, 7])

In [48]:
def create_swapped_rows(x):
    out = torch.zeros_like(x)
    out[:, 0] = x[:, 0]
    out[:, 1] = x[:, 4]
    out[:, 2] = x[:, 7]
    out[:, 3] = x[:, 3]
    out[:, 4] = x[:, 1]
    out[:, 5] = x[:, 5]
    out[:, 6] = x[:, 6]
    out[:, 7] = x[:, 2]
    return out


for x in range(8):
    print(create_swapped_rows(bb)[0,x], bb[0, x])

tensor([40.5480,  2.8542, -9.0195]) tensor([40.5480,  2.8542, -9.0195])
tensor([44.4436,  3.4255, -9.0195]) tensor([40.5480,  2.8542, -7.5801])
tensor([44.1789,  5.2300, -9.0195]) tensor([40.2834,  4.6587, -7.5801])
tensor([40.2834,  4.6587, -9.0195]) tensor([40.2834,  4.6587, -9.0195])
tensor([40.5480,  2.8542, -7.5801]) tensor([44.4436,  3.4255, -9.0195])
tensor([44.4436,  3.4255, -7.5801]) tensor([44.4436,  3.4255, -7.5801])
tensor([44.1789,  5.2300, -7.5801]) tensor([44.1789,  5.2300, -7.5801])
tensor([40.2834,  4.6587, -7.5801]) tensor([44.1789,  5.2300, -9.0195])


In [47]:
ba = example["gt_bboxes_3d"].data.corners
bb = copy.deepcopy(ba)

# swap rows in bb
bb[:, 0] = ba[:, 0]

intersection_vol, iou = box3d_overlap(create_swapped_rows(ba), create_swapped_rows(bb))
print("iou", iou)

iou tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0